PART II

This is the 2nd Jupyter notebook file. It contains the entire process of cleaning/organizing the data, and the initial training/testing to select the best-performing model. 

In [1]:
import pandas as pd
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve, validation_curve

from pathlib import Path

# classifiers we will use
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost

#imputers
from sklearn.impute import SimpleImputer, KNNImputer

# model selection bits
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, ParameterGrid, ParameterSampler
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit, GroupKFold, StratifiedKFold
from sklearn.model_selection import learning_curve, validation_curve

# evaluation
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

import scipy

import openpyxl

Again, let's load all the datasets. 

I have had an initial look at the datasets. Now the goal is to clean them up and prepare them for training models.

In [3]:
# Load the competition datasets into Pandas DataFrame
path = Path("/Users/13392/Documents/amp-parkinsons-disease-progression-prediction")
proteins = pd.read_csv(path/"train_proteins.csv")
peptides = pd.read_csv(path/"train_peptides.csv")
clinical = pd.read_csv(path/"train_clinical_data.csv")
supplemental = pd.read_csv(path/"supplemental_clinical_data.csv")

As discussed previously, we are dropping the entire "medication status" column, because:
1) Over 50% values are NaN.
2) They are non-numerical values that cannot be imputed.
2) the test dataset will not have this data. 

In [4]:
# drop the "medication status" column (due to over 50% NaN values), keep a copy of the original for later access. 
clinical_copy = clinical.copy()

clinical.drop('upd23b_clinical_state_on_medication', axis=1, inplace=True)

Here I create some lists of labels that will come handy later.

In [5]:

targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
ids = ['patient_id', 'visit_id']
month = ['visit_month']

Let's see how much remains of NaN values.

In [6]:
print(f'NaN value count:\n{clinical.isna().sum()}')
clinical

NaN value count:
visit_id          0
patient_id        0
visit_month       0
updrs_1           1
updrs_2           2
updrs_3          25
updrs_4        1038
dtype: int64


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,55,0,10.0,6.0,15.0,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0
4,55_12,55,12,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0
2611,65043_54,65043,54,4.0,8.0,11.0,1.0
2612,65043_60,65043,60,6.0,6.0,16.0,1.0
2613,65043_72,65043,72,3.0,9.0,14.0,1.0


UPDRS_4 has a significant amount of NaN values, but manageable with some kind of imputation. 

Let's count the number of visits each patient has on record and save the data in a new dataframe for later access. 

In [7]:
cols = ['patient_id', 'num_entries']
patient_list = clinical.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(clinical[clinical.patient_id==patient].index)

    n_list.append(n)
    p_list.append(patient)

df_visits_by_patient = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_visits_by_patient


,patient_id,num_entries
0,55,13
1,942,15
2,1517,10
3,1923,7
4,2660,6
...,...,...
243,63875,9
244,63889,10
245,64669,15
246,64674,16


Here I am counting NaN values in the "Proteins"  and "Peptides" datasets. 

In [8]:
print(f'NaN value count:\n{proteins.isna().sum()}')
proteins

NaN value count:
visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [9]:
print(f'NaN value count:\n{peptides.isna().sum()}')

peptides

NaN value count:
visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


Great! No NaN values at all in them. Let's find out how many of each patient's visits have protein/peptide data. 

In [10]:
cols = ['patient_id', 'num_entries_protein']
patient_list = proteins.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(proteins[proteins.patient_id==patient].visit_id.unique())

    n_list.append(n)
    p_list.append(patient)

df_recorded_visits_protein = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_recorded_visits_protein


,patient_id,num_entries_protein
0,55,4
1,1517,4
2,1923,3
3,2660,5
4,3636,3
...,...,...
243,52998,3
244,54979,3
245,58597,3
246,7508,3


In [11]:
cols = ['patient_id', 'num_entries_peptide']
patient_list = peptides.patient_id.unique()
n_list = []
p_list = []

for patient in patient_list:
    n=len(peptides[peptides.patient_id==patient].visit_id.unique())

    n_list.append(n)
    p_list.append(patient)

df_recorded_visits_peptide = pd.DataFrame(list(zip(p_list, n_list)), columns=cols)

df_recorded_visits_peptide

,patient_id,num_entries_peptide
0,55,4
1,1517,4
2,1923,3
3,2660,5
4,3636,3
...,...,...
243,52998,3
244,54979,3
245,58597,3
246,7508,3


We will combine all that into a single dataframe for better viewing.

In [12]:
df = pd.merge(df_recorded_visits_protein, df_recorded_visits_peptide, on='patient_id', how='left')
df = pd.merge(df_visits_by_patient, df, on='patient_id', how='left')
df.head(10)

,patient_id,num_entries,num_entries_protein,num_entries_peptide
0,55,13,4,4
1,942,15,4,4
2,1517,10,4,4
3,1923,7,3,3
4,2660,6,5,5
5,3636,14,3,3
6,3863,9,5,5
7,4161,12,6,6
8,4172,8,7,7
9,4923,11,5,5


It's not looking great - we can clearly see that, for most of the patients, only 1/2 to 1/3 of the visits contain protein and peptide records - this will be a problem.

Now we will use the "pivot" function, built-in for Pandas dataframes, on the datasets so the unique coding for each protein/peptide becomes a feature for the models to learn on. This might sound confusing at first, but it will quickly make sense.

In [13]:
df_proteins = proteins.pivot(index=['patient_id', 'visit_month', 'visit_id'], columns='UniProt', values='NPX').rename_axis(columns=None).reset_index()

df_peptides = peptides.pivot(index=['patient_id', 'visit_month', 'visit_id'], columns='Peptide', values='PeptideAbundance').rename_axis(columns=None).reset_index()

In [14]:
df_proteins

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,365475.0,35528.00,97005.6,23122.5,60912.6,408698.0,NaN,29758.8,23833.7,18953.5
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,405676.0,30332.60,109174.0,23499.8,51655.8,369870.0,NaN,22935.2,17722.5,16642.7
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,303953.0,43026.20,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,303597.0,48188.40,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,253373.0,27431.80,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,NaN,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,260021.0,7139.93,104277.0,10500.0,21944.2,136725.0,62217.5,NaN,10287.7,13848.2
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,186414.0,25897.80,NaN,21480.7,57364.0,416142.0,37584.6,NaN,28346.5,35617.5
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,301343.0,22343.40,105626.0,20500.8,54011.2,380072.0,40588.9,NaN,17035.7,37064.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,300439.0,52143.60,139291.0,19449.2,66569.9,300948.0,36150.4,NaN,21286.3,39587.9


In [15]:
df_peptides

,patient_id,visit_month,visit_id,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,8984260.0,53855.6,8579740.0,NaN,19735.4,114400.0,46371.1,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,8279770.0,45251.9,8655890.0,49927.5,23820.4,90539.4,38652.4,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,8382390.0,53000.9,8995640.0,45519.2,17813.5,147312.0,45840.9,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,10671500.0,58108.4,9985420.0,52374.0,19373.3,64356.1,49793.2,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,6177730.0,42682.6,3596660.0,25698.8,17130.6,86471.5,41007.9,...,226314.0,6399.80,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,7083630.0,35656.1,6273100.0,NaN,NaN,15479.2,NaN,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,7818630.0,95033.0,5119260.0,57483.7,11610.0,270739.0,42527.3,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,8070390.0,76532.7,8233520.0,54260.6,11631.9,230169.0,42255.5,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,7608150.0,75401.6,9168030.0,NaN,13313.9,220202.0,46914.1,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


Each row is now uniquely identifiable by "visit_id". There are 1113 rows (i.e. visits) with recorded proteomic data, including over 200 proteins and nearly 1000 peptides (columns). 

Because not all 1200+ proteins and peptides are measured at each recorded visit, some NaN values are now expected. Let's take a look at exactly how many for each column.

In [16]:
df_proteins.isna().sum().sort_values(ascending=False)

Q99829        624
Q99832        507
Q562R1        497
P01780        459
Q6UX71        452
             ... 
P02766          0
P02765          0
P02751          0
P02749          0
patient_id      0
Length: 230, dtype: int64

In [17]:
df_peptides.isna().sum().sort_values(ascending=False)

QALPQVR                   624
EPQVYTLPPSRDELTK          550
TPSGLYLGTC(UniMod_4)ER    523
SLEDQVEMLR                514
HYEGSTVPEK                508
                         ... 
visit_id                    0
IPTTFENGR                   0
AIGYLNTGYQR                 0
NILTSNNIDVK                 0
patient_id                  0
Length: 971, dtype: int64

We are going to combine the protein and peptide data into one dataframe (calling it: prot_pept_df), and check once again the status of NaN values. They should remain unchanged because I haven't done any modifications.

In [18]:
prot_pept_df = pd.merge(df_proteins, df_peptides, on=['patient_id','visit_month','visit_id'], how='left')
prot_pept_df

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,226314.0,6399.80,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,NaN,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


In [19]:
prot_pept_df.isna().sum().sort_values(ascending=False)

Q99829                    624
QALPQVR                   624
EPQVYTLPPSRDELTK          550
TPSGLYLGTC(UniMod_4)ER    523
SLEDQVEMLR                514
                         ... 
P41222                      0
P02774                      0
P02787                      0
P02790                      0
patient_id                  0
Length: 1198, dtype: int64

Storing a list of all unique patient ID's. I will need this later.

In [20]:
patient_list=prot_pept_df['patient_id'].unique()
patient_list

array([   55,   942,  1517,  1923,  2660,  3636,  3863,  4161,  4172,
        4923,  5027,  5036,  5178,  5645,  5742,  6054,  6211,  6420,
        7051,  7117,  7151,  7265,  7508,  7568,  7832,  7886,  8344,
        8699, 10053, 10138, 10174, 10541, 10715, 10718, 11459, 11686,
       11928, 12516, 12636, 12703, 12755, 12931, 13360, 13368, 13618,
       13804, 13852, 13968, 14035, 14124, 14242, 14270, 14344, 14450,
       14811, 15009, 15245, 15504, 15590, 16238, 16347, 16566, 16574,
       16778, 16931, 17154, 17201, 17414, 17727, 18183, 18204, 18553,
       18560, 19088, 20212, 20216, 20352, 20404, 20460, 20581, 20664,
       20707, 20791, 20792, 21126, 21537, 21729, 22126, 22623, 23175,
       23192, 23244, 23391, 23636, 24278, 24690, 24818, 24820, 24911,
       25562, 25739, 25750, 25827, 25911, 26005, 26104, 26210, 26809,
       27079, 27300, 27464, 27468, 27607, 27715, 27872, 27893, 27971,
       27987, 28327, 28342, 28818, 29313, 29417, 30119, 30155, 30416,
       30894, 30951,

Here there are two ways to impute the data:
1) combine "clinical" and "prot_pept_df" first then impute, or
2) first impute them separately, then combine. 

We'll do both and see how the outcomes differ.

I am choosing to use sklearn's KNN imputer because, with a bit of clever coding, I can use all the availble data for a given patient to impute missing values. The general rule is that, the more data there is to impute with, the better (higher confidence) the outcome.

METHOD 1: 

Combine first, impute after. 
I create a new dataframe (named "big_data") by combining dataframes "clinical" and "prot_pept_df".

In [21]:
big_data = pd.merge(clinical, prot_pept_df, on=['patient_id','visit_month','visit_id'], how='left')
big_data.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,NaN,11254.3,732430.0,39585.8,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,13163.6,630465.0,35220.8,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
3,55_9,55,9,8.0,9.0,30.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,15257.6,815083.0,41650.9,...,231772.0,17873.8,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1


Above is the combined dataset prior to KNN imputation. By counting the number of visits (num_rows) for each patient, I set the KNN Imputer to take all the visits into consideration (n_neighbors = num_rows-1). Setting "weights='distance'" allows the imputer to weigh closer neighboring values more heavily than more distant values. This makes sense medically as well, as chronic diseases tend to not have drastic fluctuations in both blood tests and CSF measurements. Our data is arranged chronologically, hence the closer neighboring data should be more similar. 

In [22]:
#imputing
big_data_patient_list = big_data.patient_id.unique()
data_imputed_list = []
for patient_id in big_data_patient_list:
    masked_data = big_data[big_data['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows-1, weights='distance')
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = big_data.columns)
    data_imputed_list.append(X_knn_df)

big_data_imputed = pd.concat(data_imputed_list, ignore_index=True)

Here is "big_data" after KNN imputation, from here on labeled "big_data_imputed."

In [23]:
big_data_imputed

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,550.0,55.0,0.0,10.0,6.0,15.0,0.000000,11254.300000,732430.000000,39585.800000,...,201158.000000,16492.300000,3.810270e+06,106894.000000,580667.000000,131155.000000,165851.000000,437305.000000,46289.200000,14898.400000
1,553.0,55.0,3.0,10.0,7.0,25.0,0.000000,12334.785768,675214.704118,37137.660530,...,184262.267618,14644.342049,3.984725e+06,110536.418287,543921.541924,115616.062135,153965.859351,448964.062600,42796.419382,18169.287093
2,556.0,55.0,6.0,8.0,10.0,34.0,0.000000,13163.600000,630465.000000,35220.800000,...,171079.000000,13198.800000,4.119520e+06,113385.000000,514861.000000,103512.000000,144607.000000,457891.000000,40047.700000,20703.900000
3,559.0,55.0,9.0,8.0,9.0,30.0,0.000000,12862.372461,646873.277154,35923.996315,...,175907.727634,13728.866925,4.070366e+06,112340.558877,525569.281503,107942.057630,148048.968011,454649.313064,41058.346150,19778.422756
4,5512.0,55.0,12.0,10.0,10.0,41.0,0.000000,15257.600000,815083.000000,41650.900000,...,231772.000000,17873.800000,5.474140e+06,116286.000000,711815.000000,136943.000000,181763.000000,452253.000000,54725.100000,21841.100000
5,5518.0,55.0,18.0,7.0,13.0,38.0,0.000000,14986.051248,805405.447723,41858.782472,...,224527.244180,17977.316686,5.037707e+06,112368.453483,706490.758504,135621.228578,185093.414028,459390.594547,54406.710646,20621.446960
6,5524.0,55.0,24.0,16.0,9.0,49.0,0.000000,14425.806933,785489.935290,42300.996496,...,209508.095440,18199.369402,4.130422e+06,104209.424406,695717.193668,132903.900331,192093.918346,474288.047929,53765.988135,18085.590942
7,5530.0,55.0,30.0,14.0,13.0,49.0,0.000000,14050.064768,772176.096980,42608.884997,...,199373.871407,18355.493866,3.516106e+06,98672.385379,688670.794502,131089.667780,196898.785238,484425.006370,53349.986393,16368.275124
8,5536.0,55.0,36.0,17.0,18.0,51.0,0.000000,13530.800000,753832.000000,43048.900000,...,185290.000000,18580.500000,2.659660e+06,90936.900000,679163.000000,128593.000000,203680.000000,498621.000000,52792.700000,13973.700000
9,5542.0,55.0,42.0,12.0,20.0,41.0,0.000000,13968.772329,769262.341576,42666.746983,...,197228.716914,18383.700115,3.387703e+06,97524.725463,687007.633880,130690.799535,197853.349919,486505.470464,53249.369332,16009.561976


Formats of the columns "visit_id", "patient_id", "visit_month" have been changed in the process of imputation, so they need to be reset. 

In [24]:
#relabel patient id, visit id, and visit month.
ls = [ids, month]
for col in ls:
    big_data_imputed[col] = big_data[col]

In [25]:
big_data_imputed.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,0.0,11254.300000,732430.000000,39585.800000,...,201158.000000,16492.300000,3.810270e+06,106894.000000,580667.000000,131155.000000,165851.000000,437305.000000,46289.200000,14898.400000
1,55_3,55,3,10.0,7.0,25.0,0.0,12334.785768,675214.704118,37137.660530,...,184262.267618,14644.342049,3.984725e+06,110536.418287,543921.541924,115616.062135,153965.859351,448964.062600,42796.419382,18169.287093
2,55_6,55,6,8.0,10.0,34.0,0.0,13163.600000,630465.000000,35220.800000,...,171079.000000,13198.800000,4.119520e+06,113385.000000,514861.000000,103512.000000,144607.000000,457891.000000,40047.700000,20703.900000
3,55_9,55,9,8.0,9.0,30.0,0.0,12862.372461,646873.277154,35923.996315,...,175907.727634,13728.866925,4.070366e+06,112340.558877,525569.281503,107942.057630,148048.968011,454649.313064,41058.346150,19778.422756
4,55_12,55,12,10.0,10.0,41.0,0.0,15257.600000,815083.000000,41650.900000,...,231772.000000,17873.800000,5.474140e+06,116286.000000,711815.000000,136943.000000,181763.000000,452253.000000,54725.100000,21841.100000


In [65]:
big_data_imputed.shape

(2615, 1202)

Note the size of the dataframe "big_data_imputed": 2615 by 1202. 

Next is METHOD #2: impute each dataset first, then combine. 

Imputing prot_pept_df:

In [26]:
data_imputed_list = []
for patient_id in patient_list:
    masked_data = prot_pept_df[prot_pept_df['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows, weights='distance')
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = prot_pept_df.columns)
    data_imputed_list.append(X_knn_df)
prot_pept_imputed = pd.concat(data_imputed_list, ignore_index=True)

There should be no NaN values left in the dataframe.

In [27]:
prot_pept_imputed.isna().sum()

patient_id               0
visit_month              0
visit_id                 0
O00391                   0
O00533                   0
                        ..
YVNKEIQNAVNGVK           0
YWGVASFLQK               0
YYC(UniMod_4)FQGNQFLR    0
YYTYLIMNK                0
YYWGGQYTWDMAK            0
Length: 1198, dtype: int64

Resetting "patient_id", "visit_month" and "visit_id" back to the original data types and formats.

In [28]:
prot_pept_imputed = prot_pept_imputed.astype({'patient_id': 'int', 'visit_month': 'int'})
prot_pept_imputed['visit_id'] = prot_pept_df['visit_id']
print(prot_pept_imputed.dtypes)

patient_id                 int32
visit_month                int32
visit_id                  object
O00391                   float64
O00533                   float64
                          ...   
YVNKEIQNAVNGVK           float64
YWGVASFLQK               float64
YYC(UniMod_4)FQGNQFLR    float64
YYTYLIMNK                float64
YYWGGQYTWDMAK            float64
Length: 1198, dtype: object


In [29]:
prot_pept_imputed

,patient_id,visit_month,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55,0,55_0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,...,201158.0,16492.30,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55,6,55_6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,...,171079.0,13198.80,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55,12,55_12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,...,231772.0,17873.80,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55,36,55_36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,...,185290.0,18580.50,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942,6,942_6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,...,226314.0,6399.80,374307.0,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,64674_84,0.0,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,...,203523.0,3835.58,4901220.0,40325.9,335625.0,49250.4,64076.3,667993.0,38472.5,21949.1
1109,65043,0,65043_0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,...,257361.0,18316.60,2514660.0,51444.6,530245.0,156148.0,157548.0,336625.0,48423.2,10915.8
1110,65043,12,65043_12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,...,230437.0,16703.20,2481560.0,44405.0,543391.0,159828.0,161207.0,330337.0,45368.1,19023.2
1111,65043,24,65043_24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,...,251228.0,18326.20,2939460.0,50588.2,597869.0,148032.0,192857.0,388125.0,65101.0,20790.1


The proteomic dataframe "prot_pept_df" is now imputed, stored as "prot_pept_imputed". Note that the dimension is 1113 rows by 1198 columns. 

Next up is dataframe "clinical'.

In [30]:
print(clinical.isna().sum())
print(f'dataset shape: {clinical.shape}')

visit_id          0
patient_id        0
visit_month       0
updrs_1           1
updrs_2           2
updrs_3          25
updrs_4        1038
dtype: int64
dataset shape: (2615, 7)


Imputing dataframe "clinical":

In [31]:
data_imputed_list = []
for patient_id in patient_list:
    masked_data = clinical[clinical['patient_id']==patient_id]
    num_rows = len(masked_data.index)
    knn = KNNImputer(missing_values=np.nan, keep_empty_features=True, n_neighbors=num_rows-1, weights='distance')
    X_knn = knn.fit_transform(masked_data)
    X_knn_df = pd.DataFrame(X_knn, columns = clinical.columns)
    data_imputed_list.append(X_knn_df)
clinical_imputed = pd.concat(data_imputed_list, ignore_index=True)

Showing that imputation has been successfully applied, as there are no NaN values left in the dataframe. 

In [32]:
print(clinical_imputed.isna().sum())
print(f'imputed clinical dataset shape: {clinical_imputed.shape}')

visit_id       0
patient_id     0
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
dtype: int64
imputed clinical dataset shape: (2615, 7)


The dimension is 2615 rows by 7 columns.

Similarly to what I have done previously, here I am resetting the formatting of the columns that were changed in the imputing process.

In [33]:
clinical_imputed = clinical_imputed.astype({'patient_id': 'int', 'visit_month': 'int'})
clinical_imputed['visit_id'] = clinical['visit_id']
print(clinical_imputed.dtypes)

visit_id        object
patient_id       int32
visit_month      int32
updrs_1        float64
updrs_2        float64
updrs_3        float64
updrs_4        float64
dtype: object


Now the final step of METHOD 2 is to merge the 2 imputed dataframes, storing them in one dataframe labeled "method_2_imputed". Merging on ['visit_id', 'visit_month', 'patient_id'] would mean that any rows that don't have overlapping values in these 3 columns would simply be lost. Hence, one would expect the number of rows in the final dataset to be that of the lesser of two, i.e. 1113 rows. 

In [34]:

method_2_imputed = pd.merge(prot_pept_imputed, clinical_imputed, on=['visit_id', 'visit_month', 'patient_id'], how='left')


In [35]:
print(method_2_imputed.isna().sum())
print(f'merged protein peptide and clinical dataset shape: {method_2_imputed.shape}')

patient_id        0
visit_month       0
visit_id          0
O00391            0
O00533            0
                 ..
YYWGGQYTWDMAK     0
updrs_1          45
updrs_2          45
updrs_3          45
updrs_4          45
Length: 1202, dtype: int64
merged protein peptide and clinical dataset shape: (1113, 1202)


Indeed, I end up with a dataset that is 1113 rows by 1202 columns. 

Interestingly, with Method #2 (first impute separately, then combine) we end up with some additional NaN values that need to be dealt with. 

In [36]:
df = method_2_imputed[method_2_imputed.columns[method_2_imputed.isna().any()]]

df1 = df[df.isna().any(axis=1)]

print(df1)
print()
print(f'Remaining NaN values is a dataframe of: {df1.shape}')

      updrs_1  updrs_2  updrs_3  updrs_4
16        NaN      NaN      NaN      NaN
47        NaN      NaN      NaN      NaN
50        NaN      NaN      NaN      NaN
57        NaN      NaN      NaN      NaN
92        NaN      NaN      NaN      NaN
95        NaN      NaN      NaN      NaN
156       NaN      NaN      NaN      NaN
172       NaN      NaN      NaN      NaN
182       NaN      NaN      NaN      NaN
216       NaN      NaN      NaN      NaN
219       NaN      NaN      NaN      NaN
239       NaN      NaN      NaN      NaN
269       NaN      NaN      NaN      NaN
338       NaN      NaN      NaN      NaN
342       NaN      NaN      NaN      NaN
364       NaN      NaN      NaN      NaN
378       NaN      NaN      NaN      NaN
405       NaN      NaN      NaN      NaN
406       NaN      NaN      NaN      NaN
411       NaN      NaN      NaN      NaN
431       NaN      NaN      NaN      NaN
439       NaN      NaN      NaN      NaN
474       NaN      NaN      NaN      NaN
499       NaN   

In the case of these rows that contain NaN values, all 4 parts of the UPDRS score are empty. This means that, during these visits, although the proteomic data is collected, UPDRS scores were not recorded, rendering these visits useless, for all practial purposes. I decide that the best course of action here would be to drop the 45 rows entirely. 

In [37]:
method_2_imputed.dropna(subset = targets, inplace=True, axis=0)
method_2_imputed.isna().sum()

patient_id       0
visit_month      0
visit_id         0
O00391           0
O00533           0
                ..
YYWGGQYTWDMAK    0
updrs_1          0
updrs_2          0
updrs_3          0
updrs_4          0
Length: 1202, dtype: int64

Here is a quick comparison of the dimensions of the two imputed datasets from our 2 different methods. 

In [64]:
print(f'big_data_imputed df dimension: {big_data_imputed.shape}')
print(f'method_2_imputed df dimension: {method_2_imputed.shape}')

big_data_imputed df dimension: (2615, 1202)
method_2_imputed df dimension: (1068, 1202)


big_data_imputed df dimension: (2615, 1202)

method_2_imputed df dimension: (1068, 1202)

Essentially, the first method yielded a dataset ('big_data_imputed') that is nearly 2.5 times the amount of the 2nd method ('method_2_imputed'). Judging just by size, the first method should produce models that score sigificantly better than the other. 

But before introducing ML models, I'll first define a function to calculate SMAPE scores. 

In [38]:
def smape_score(actual, predicted):
    sum = 0
    for a, p in zip(actual, predicted):
        if a==0 and p==0:
            pass
        else:
            sum += (np.abs(p-a))/(np.abs(p)+np.abs(a))*2
    return sum/len(actual)*100

The models I decide to test include:

Random Forest Regressor, HistGradientBoostingRegressor, Tree based AdaBoostRegressor, Tree based XGBRegressor.


Before specifying parameters for each model, my first question is: if I don't specify any parameters, what does the model do on its own? 

Let's first take a look at RandomForestRegressor training on 'big_data_imputed'. 

In [39]:
RFR = RandomForestRegressor() # not specifying any parameters.

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    #fitting and testing
    RFR.fit(X_train,y_train)    
    y_predict = RFR.predict(X_test)
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving the scores in respective lists
    print(f'MAE: {mae}; SMAPE: {smape}')
    print()

    max_depth = list()

    for tree in RFR.estimators_:
        max_depth.append(tree.tree_.max_depth)

    print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
    print(f"max depth {max(max_depth)}")
    print(f"min depth {min(max_depth)}")
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
MAE: 2.463920909249787; SMAPE: 48.62677488088776

avg max depth 26.6
max depth 33
min depth 20

updrs_2: 2353 samples in training, 262 samples in testing.
MAE: 2.357862595366331; SMAPE: 66.6338791862446

avg max depth 26.0
max depth 37
min depth 18

updrs_3: 2353 samples in training, 262 samples in testing.
MAE: 6.078327276829435; SMAPE: 58.1046977048397

avg max depth 23.4
max depth 35
min depth 18

updrs_4: 2353 samples in training, 262 samples in testing.
MAE: 0.9512113206857896; SMAPE: 137.55895110590444

avg max depth 57.8
max depth 81
min depth 32



In [40]:
depth, counts = np.unique(max_depth, return_counts=True)
print(f'unique depths: {depth}')
print(f'occurances: {counts}')

unique depths: [32 36 38 40 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 61 62
 63 64 65 66 67 68 69 70 71 73 74 81]
occurances: [1 1 1 1 1 1 1 2 1 2 3 3 1 5 2 3 7 7 4 3 7 4 2 4 1 7 5 3 2 3 2 2 2 2 3 1]


A random forest regressor with no specified parameters, training on this dataset, reaches a minimum depth of 32 and a maximum depth of 81, which means the models appears to favor deep trees with this dataset - an important feature to keep in mind. 

For RandomForestRegressor, argurably the most important parameters to control are:

n_estimators, the number of decision trees in the forest;

max_depth, the max number of splits that each tree is allowed.

I'll select some parameters, based on feedback from the previous uncontrolled RandomForest Regressor, which noticeably favors deeper trees for this dataset. So I will set a range of max_depth from 5 to 50, and n_estimators (number of trees in the forest) from 100 (default) to 800. I'll then compare these models to the default model.

In [41]:

RFR = RandomForestRegressor()

params = {'n_estimators': [100,200,500,800],
                 'max_depth': [5,10,20,30,40,50]}

random_params = list(ParameterSampler(params, n_iter=5, random_state=1))

random_params


[{'n_estimators': 200, 'max_depth': 30},
 {'n_estimators': 500, 'max_depth': 40},
 {'n_estimators': 800, 'max_depth': 5},
 {'n_estimators': 500, 'max_depth': 30},
 {'n_estimators': 100, 'max_depth': 50}]

I'll first save these pseudo randomly generated parameters in a dataframe, for later referencing.

In [42]:
params_df = pd.DataFrame(random_params)
params_df

,n_estimators,max_depth
0,200,30
1,500,40
2,800,5
3,500,30
4,100,50


I'll first test the RandomForestRegressor model on "big_data_imputed", i.e. data that was first combined then imputed. 



In [43]:
#randomforestregressor 
smape_list_RFR1 = []
mae_list_RFR1 = []
actual_depth_RFR1 = []

#defining features
X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:
    #setting target variable
    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        RFR.set_params(**params)
        RFR.fit(X_train,y_train)    
        y_predict = RFR.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving the scpres in respective lists
        mae_list_RFR1.append(mae)
        smape_list_RFR1.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')
        #still good to know how deep the trees are actually going.
        max_depth = list()
        for tree in RFR.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_RFR1.append(max(max_depth))
    print()


updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.418836701171328; SMAPE: 47.97181361660714
avg max depth 26.8
max depth 30
min depth 21

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.424152130011467; SMAPE: 48.1700077388935
avg max depth 27.1
max depth 37
min depth 21

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 3.279790074041403; SMAPE: 57.4363676661162
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.44353142398715; SMAPE: 48.18737954653573
avg max depth 26.8
max depth 30
min depth 20

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.426192240206361; SMAPE: 47.954575747798195
avg max depth 26.4
max depth 37
min depth 21


updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.3237237709211054; SMAPE: 68.34096625852366
avg max depth 25.3
max depth 30
min depth 19

Parameter:

I will store the scores and their respective parameters in a dataframe, for easy referencing. 

In [44]:
RFR_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
RFR_df['Actual Depth Reached'] = actual_depth_RFR1
RFR_df['MAE'] = mae_list_RFR1
RFR_df['SMAPE'] = smape_list_RFR1
RFR_df['UPDRS'] = ""
RFR_df.loc[:4, 'UPDRS']=1
RFR_df.loc[5:9, 'UPDRS']=2
RFR_df.loc[10:14, 'UPDRS']=3
RFR_df.loc[15:, 'UPDRS']=4

RFR_df

,n_estimators,max_depth,Actual Depth Reached,MAE,SMAPE,UPDRS
0,200,30,30,2.418837,47.971814,1
1,500,40,37,2.424152,48.170008,1
2,800,5,5,3.279790,57.436368,1
3,500,30,30,2.443531,48.187380,1
4,100,50,37,2.426192,47.954576,1
5,200,30,30,2.323724,68.340966,2
6,500,40,40,2.365266,68.472014,2
7,800,5,5,3.809292,80.389840,2
8,500,30,30,2.363213,68.402228,2
9,100,50,37,2.341947,65.964504,2


I am exporting the results to an Excel file for later referencing. 

In [45]:
RFR_df.to_excel('initial_model_selection_results.xlsx', sheet_name='RFR') # saving the results in an excel file.

Note that the lowest MAE scores may not correspond with the lowest SMAPE scores, but I have already discussed and established that SMAPE may not be the best metric for this model. Mean Absolute Error, on the other hand, paints a clearer picture.

Also note that the training dataset contains 2353 samples, and the testing dataset contains 262 samples. 

Now we'll test the same set of parameters on the same model, on "method_2_imputed" data, i.e. data that was first independently imputed, then combined. Like I have previously pointed out, this method produces a dataset that it less than 50% the size of Method 1. I would expect the trained model to perform more poorly. 

In [46]:
#defining features
X = method_2_imputed.drop(columns=targets, axis=1)
smape_list_RFR2 = []
mae_list_RFR2 = []
actual_depth_RFR2 = []

for target in targets:
    #setting target variable
    y = method_2_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        RFR.set_params(**params)
        RFR.fit(X_train,y_train)    
        y_predict = RFR.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving the scpres in respective lists
        mae_list_RFR2.append(mae)
        smape_list_RFR2.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')
        #still good to know how deep the trees are actually going.
        max_depth = list()
        for tree in RFR.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_RFR2.append(max(max_depth))

    print()

updrs_1: 961 samples in training, 107 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.958136968172015; SMAPE: 60.92895779752984
avg max depth 22.9
max depth 30
min depth 16

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.9008598130841126; SMAPE: 60.12616219809903
avg max depth 22.8
max depth 34
min depth 15

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.9591347717700263; SMAPE: 60.694796436413924
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.9172947542958214; SMAPE: 60.11579062932542
avg max depth 22.9
max depth 30
min depth 16

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.867289719626168; SMAPE: 59.72654468529359
avg max depth 22.1
max depth 31
min depth 16


updrs_2: 961 samples in training, 107 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 3.1993597086671373; SMAPE: 83.82154306732666
avg max depth 22.3
max depth 30
min depth 14

Parame

In [47]:
RFR_df_2 = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
#RFR_df_2['Actual Depth Reached'] = actual_depth_RFR2
RFR_df_2['MAE'] = mae_list_RFR2
RFR_df_2['SMAPE'] = smape_list_RFR2
RFR_df_2['UPDRS'] = ""
RFR_df_2.loc[:4, 'UPDRS']=1
RFR_df_2.loc[5:9, 'UPDRS']=2
RFR_df_2.loc[10:14, 'UPDRS']=3
RFR_df_2.loc[15:, 'UPDRS']=4

RFR_df_2

,n_estimators,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.958137,60.928958,1
1,500,40,2.900860,60.126162,1
2,800,5,2.959135,60.694796,1
3,500,30,2.917295,60.115791,1
4,100,50,2.867290,59.726545,1
5,200,30,3.199360,83.821543,2
6,500,40,3.173028,83.856774,2
7,800,5,3.480850,86.712264,2
8,500,30,3.101139,82.723526,2
9,100,50,3.137944,82.961445,2


Overall, the model performs significantly better across all tested parameters with the first dataset ("big_data_imputed"). This is as expected, since it contains nearly 2.5 times the data as this second dataset ("method_2_imputed"). i will forgo the initial random parameter search with this smaller dataset on all the other models from now on.

The next model is the Histogram Gradient Boosting Regressor (HGBR). 

I'll first test HGBR without specifying parameters, and see where the dataset takes the model. 

Note: 

1) n_estimators (number of trees) is labeled as "max_iter" for HGBR. 

2) Unfortunately there is no easy way to extract information from HGBR like the exact depth reached by each tree, like I have previously done with RandomForestRegressor. So I'll use the same randomly generated parameters previously.

In [49]:
hgb = HistGradientBoostingRegressor()
print(hgb.get_params())
X = big_data_imputed.drop(columns=targets, axis=1)


for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=10)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')

    #Randomly generate a dictionary containing 10 combinations of parameters. 
    
    #fitting and testing
    hgb.fit(X_train,y_train)    
    y_predict = hgb.predict(X_test)
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving scores in their lists
    print(f'MAE: {mae}; SMAPE: {smape}')
    #finding actual tested parameters
    print(f'Number of iterations: {hgb.n_iter_}')
    print(f'number of trees per iteration: {hgb.n_trees_per_iteration_} ')
    print()

{'categorical_features': None, 'early_stopping': 'auto', 'interaction_cst': None, 'l2_regularization': 0.0, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_bins': 255, 'max_depth': None, 'max_iter': 100, 'max_leaf_nodes': 31, 'min_samples_leaf': 20, 'monotonic_cst': None, 'n_iter_no_change': 10, 'quantile': None, 'random_state': None, 'scoring': 'loss', 'tol': 1e-07, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
updrs_1: 2353 samples in training, 262 samples in testing.
MAE: 2.346062206994689; SMAPE: 52.02608477275261
Number of iterations: 100
number of trees per iteration: 1 

updrs_2: 2353 samples in training, 262 samples in testing.
MAE: 2.1736857180719236; SMAPE: 67.91874909798729
Number of iterations: 100
number of trees per iteration: 1 

updrs_3: 2353 samples in training, 262 samples in testing.
MAE: 5.8957439955348425; SMAPE: 55.12563046296315
Number of iterations: 100
number of trees per iteration: 1 

updrs_4: 2353 samples in training, 262 samples in test

Nothing unexpected here - I am going to set the same parameter grid as previously done with a Random Forest Regressor.

In [50]:
params_HGBR = {'max_iter': [100,200,500,800],
                 'max_depth': [5,10,20,30,40,50]}
random_params_HGBR = list(ParameterSampler(params_HGBR, n_iter=5, random_state=1))

#save them in a dataframe
params_df_hgb = pd.DataFrame(random_params_HGBR)
params_df_hgb

,max_iter,max_depth
0,200,30
1,500,40
2,800,5
3,500,30
4,100,50


In [51]:
#HistGradientBoostingRegressor with big_data_imputed

hgb = HistGradientBoostingRegressor()

X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_hgb = []
mae_list_hgb = []

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')

    #Randomly generate a dictionary containing 10 combinations of parameters. 
    
    for params in random_params_HGBR:
        #fitting and testing
        hgb.set_params(**params)
        hgb.fit(X_train,y_train)    
        y_predict = hgb.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving scores in their lists
        mae_list_hgb.append(mae)
        smape_list_hgb.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')

    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'max_iter': 200, 'max_depth': 30}; MAE: 2.286743887513747; SMAPE: 47.82956439354023
Parameter: {'max_iter': 500, 'max_depth': 40}; MAE: 2.3248869501974516; SMAPE: 48.74306021770098
Parameter: {'max_iter': 800, 'max_depth': 5}; MAE: 2.4331230374677535; SMAPE: 50.32746776849654
Parameter: {'max_iter': 500, 'max_depth': 30}; MAE: 2.3248869501974516; SMAPE: 48.74306021770098
Parameter: {'max_iter': 100, 'max_depth': 50}; MAE: 2.2485680140694457; SMAPE: 46.82907006760175

updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'max_iter': 200, 'max_depth': 30}; MAE: 2.182410343595119; SMAPE: 68.02941752361711
Parameter: {'max_iter': 500, 'max_depth': 40}; MAE: 2.2096857934444873; SMAPE: 68.78954493371981
Parameter: {'max_iter': 800, 'max_depth': 5}; MAE: 2.246626373207976; SMAPE: 68.9096768398202
Parameter: {'max_iter': 500, 'max_depth': 30}; MAE: 2.2096857934444873; SMAPE: 68.78954493371981
Parameter: {'m

Like I've done previously, I will combine the parameters and their scores into a single dataframe for easy referencing.

In [52]:
HGBR_df = pd.concat([params_df_hgb,params_df_hgb,params_df_hgb,params_df_hgb], ignore_index=True)
HGBR_df['MAE'] = mae_list_hgb
HGBR_df['SMAPE'] = smape_list_hgb
HGBR_df['UPDRS'] = ""
HGBR_df.loc[:4, 'UPDRS']=1
HGBR_df.loc[5:9, 'UPDRS']=2
HGBR_df.loc[10:14, 'UPDRS']=3
HGBR_df.loc[15:, 'UPDRS']=4
HGBR_df

,max_iter,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.286744,47.829564,1
1,500,40,2.324887,48.743060,1
2,800,5,2.433123,50.327468,1
3,500,30,2.324887,48.743060,1
4,100,50,2.248568,46.829070,1
5,200,30,2.182410,68.029418,2
6,500,40,2.209686,68.789545,2
7,800,5,2.246626,68.909677,2
8,500,30,2.209686,68.789545,2
9,100,50,2.198087,68.014661,2


I am adding the HGBR results to the Excel file. Note that SMAPE scores for UPDRS_4 are significantly worse - this is the same outcome as the Random Forest Regressor.

In [53]:
HGBR_df.to_excel('initial_model_selection_results.xlsx', sheet_name='HGBR')

Now onto AdaBoost Regressor with Decision Tree base regressor.

Again, first I train and test the model without any preset parameters.

In [54]:
#ADABOOST REGRESSOR with decision tree regressor
X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_ada = []
mae_list_ada = []

for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    

    #fitting and testing
    ada = AdaBoostRegressor(DecisionTreeRegressor())
    print(ada.get_params())
    ada.fit(X_train,y_train)    
    y_predict = ada.predict(X_test)

    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    print(f'MAE: {mae}; SMAPE: {smape}')

    #saving scores in their lists
    mae_list_ada.append(mae)
    smape_list_ada.append(smape)

    max_depth = list()
    for tree in ada.estimators_:
        max_depth.append(tree.tree_.max_depth)
    print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
    print(f"max depth {max(max_depth)}")
    print(f"min depth {min(max_depth)}")
    print()

    print()

updrs_1: 2353 samples in training, 262 samples in testing.
{'base_estimator': 'deprecated', 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__random_state': None, 'estimator__splitter': 'best', 'estimator': DecisionTreeRegressor(), 'learning_rate': 1.0, 'loss': 'linear', 'n_estimators': 50, 'random_state': None}
MAE: 2.3396946564885495; SMAPE: 47.970568373524166
avg max depth 24.3
max depth 30
min depth 18


updrs_2: 2353 samples in training, 262 samples in testing.
{'base_estimator': 'deprecated', 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'es

The trees are reaching a maximum depth of 66 and minimum depth of 17. What is very interesting here is that the SMAPE scores for UPDRS_4 are by far the best scores we have seen so far! In fact, all scores across the board are much better. 

I am going to further explore this model with a parameter grid, using the same parameters as previous models. 

In [55]:
random_params

[{'n_estimators': 200, 'max_depth': 30},
 {'n_estimators': 500, 'max_depth': 40},
 {'n_estimators': 800, 'max_depth': 5},
 {'n_estimators': 500, 'max_depth': 30},
 {'n_estimators': 100, 'max_depth': 50}]

In [56]:
#ADABOOST REGRESSOR with decision tree regressor
X = big_data_imputed.drop(columns=targets, axis=1)

smape_list_ada = []
mae_list_ada = []
actual_depth_ada = []


for target in targets:

    y = big_data_imputed[target]
    
    #splitting training and testing data. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    for params in random_params:
        #fitting and testing
        dtr = DecisionTreeRegressor(max_depth=params['max_depth'])
        ada = AdaBoostRegressor(dtr, n_estimators=params['n_estimators'])
        ada.fit(X_train,y_train)    
        y_predict = ada.predict(X_test)

        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')

        #saving scores in their lists
        mae_list_ada.append(mae)
        smape_list_ada.append(smape)

        max_depth = list()
        for tree in ada.estimators_:
            max_depth.append(tree.tree_.max_depth)
        print("avg max depth %0.1f" % (sum(max_depth) / len(max_depth)))
        print(f"max depth {max(max_depth)}")
        print(f"min depth {min(max_depth)}")
        print()
        actual_depth_ada.append(max(max_depth))
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.33587786259542; SMAPE: 48.09241892706094
avg max depth 23.9
max depth 30
min depth 18

Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.351145038167939; SMAPE: 47.94659477400033
avg max depth 24.1
max depth 35
min depth 17

Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.7177112386740374; SMAPE: 51.77990703630927
avg max depth 5.0
max depth 5
min depth 5

Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.33587786259542; SMAPE: 47.354430717014075
avg max depth 23.9
max depth 30
min depth 17

Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.3625954198473282; SMAPE: 47.91677067772064
avg max depth 23.4
max depth 39
min depth 18


updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.1565207588494864; SMAPE: 49.7072461090993
avg max depth 22.8
max depth 30
min depth 17

Paramete

In [57]:
ada_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
ada_df['MAE'] = mae_list_ada
ada_df['SMAPE'] = smape_list_ada
ada_df['actual depth reached'] = actual_depth_ada
ada_df['UPDRS'] = ""
ada_df.loc[:4, 'UPDRS']=1
ada_df.loc[5:9, 'UPDRS']=2
ada_df.loc[10:14, 'UPDRS']=3
ada_df.loc[15:, 'UPDRS']=4

ada_df

,n_estimators,max_depth,MAE,SMAPE,actual depth reached,UPDRS
0,200,30,2.335878,48.092419,30,1
1,500,40,2.351145,47.946595,35,1
2,800,5,2.717711,51.779907,5,1
3,500,30,2.335878,47.354431,30,1
4,100,50,2.362595,47.916771,39,1
5,200,30,2.156521,49.707246,30,2
6,500,40,2.175573,50.443636,35,2
7,800,5,3.121856,74.170145,5,2
8,500,30,2.148855,49.866082,30,2
9,100,50,2.125954,46.280116,33,2


All the scores (both MAE and SMAPE) look amazing across the board, except for a few instances where the max_depth=5. Those models performed poorly. This is expected - we have already known that our tree-based regressors prefer deeper trees for our dataset. 

In [58]:
ada_df.to_excel('initial_model_selection_results.xlsx', sheet_name='ADA')

XGBoost Regressor is the last model to test. 

In [59]:
xgbr = xgboost.XGBRegressor()
smape_list_xgb = []
mae_list_xgb = []

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='visit_id'), y, train_size=0.9, random_state=100) 

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
    
    #fitting and testing
    xgbr.fit(X_train, y_train)
    y_predict = xgbr.predict(X_test)
    #print parameters
    print(f'xgbr.get_params(): {xgbr.get_params()}')
    print(f'xgbr.get_xgb_params(): {xgbr.get_xgb_params()}')
    #scoring
    mae = mean_absolute_error(y_test, y_predict)
    smape = smape_score(y_test, y_predict)
    #saving scores in their lists
    mae_list_xgb.append(mae)
    smape_list_xgb.append(smape)
    print(f'MAE: {mae}; SMAPE: {smape}')
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
xgbr.get_params(): {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
xgbr.get_xgb_params(): {'ob

No notable information here, except that SMAPE scores for UPDRS_4 are pretty terrible again. I will use the same set of "random_params" for a more thorough test.

In [60]:
random_params

[{'n_estimators': 200, 'max_depth': 30},
 {'n_estimators': 500, 'max_depth': 40},
 {'n_estimators': 800, 'max_depth': 5},
 {'n_estimators': 500, 'max_depth': 30},
 {'n_estimators': 100, 'max_depth': 50}]

In [61]:
#Tree based XGB Regressor using big_data_imputed

xgbr = xgboost.XGBRegressor()

smape_list_xgb = []
mae_list_xgb = []

X = big_data_imputed.drop(columns=targets, axis=1)

for target in targets:

    y = big_data_imputed[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='visit_id'), y, train_size=0.9, random_state=100) 

    print(f'{target}: {len(X_train)} samples in training, {len(X_test)} samples in testing.')
   
    for params in random_params:
        #fitting and testing
        xgbr.set_params(**params)
        xgbr.fit(X_train, y_train)
        y_predict = xgbr.predict(X_test)
        #scoring
        mae = mean_absolute_error(y_test, y_predict)
        smape = smape_score(y_test, y_predict)
        #saving scores in their lists
        mae_list_xgb.append(mae)
        smape_list_xgb.append(smape)
        print(f'Parameter: {params}; MAE: {mae}; SMAPE: {smape}')      
    print()

updrs_1: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.40968325038357; SMAPE: 47.61712181500031
Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.4028405244355553; SMAPE: 47.61969153699509
Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.345453493126476; SMAPE: 47.267006209222906
Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.4096832516534743; SMAPE: 47.61712181500031
Parameter: {'n_estimators': 100, 'max_depth': 50}; MAE: 2.400939074232961; SMAPE: 47.64621458948384

updrs_2: 2353 samples in training, 262 samples in testing.
Parameter: {'n_estimators': 200, 'max_depth': 30}; MAE: 2.111496996942373; SMAPE: 65.13391478579933
Parameter: {'n_estimators': 500, 'max_depth': 40}; MAE: 2.12273758448793; SMAPE: 64.92593173512918
Parameter: {'n_estimators': 800, 'max_depth': 5}; MAE: 2.171012562986211; SMAPE: 67.22931763756247
Parameter: {'n_estimators': 500, 'max_depth': 30}; MAE: 2.1114969969400956; SMAPE:

In [62]:
xgb_df = pd.concat([params_df,params_df,params_df,params_df], ignore_index=True)
xgb_df['MAE'] = mae_list_xgb
xgb_df['SMAPE'] = smape_list_xgb
xgb_df['UPDRS'] = ""
xgb_df.loc[:4, 'UPDRS']=1
xgb_df.loc[5:9, 'UPDRS']=2
xgb_df.loc[10:14, 'UPDRS']=3
xgb_df.loc[15:, 'UPDRS']=4

xgb_df

,n_estimators,max_depth,MAE,SMAPE,UPDRS
0,200,30,2.409683,47.617122,1
1,500,40,2.402841,47.619692,1
2,800,5,2.345453,47.267006,1
3,500,30,2.409683,47.617122,1
4,100,50,2.400939,47.646215,1
5,200,30,2.111497,65.133915,2
6,500,40,2.122738,64.925932,2
7,800,5,2.171013,67.229318,2
8,500,30,2.111497,65.133915,2
9,100,50,2.121559,64.929670,2


In [63]:
xgb_df.to_excel('initial_model_selection_results.xlsx', sheet_name='XGBR')

Similarly, XGBoost Regressor performs decently well, except on predicting UPDRS_4 scores. The SMAPE scores are calculated as percentage error scores, which means although the MAE scores are actually relatively low (less than 1), the percentage error is much higher because the actual UPDRS_4 scores are so low. 

Out of the 4 models tested, only AdaBoost Regressor performs impressively well across all UPDRS scores - as well as the best of the other models in UPDRS_1, _2 and _3, and significantly better in _4. One could spend a lot more time fine-tuning the parameters on all the models to be definitively sure, but here I already have reasonable certainty that an AdaBoost Regressor would be a great choice to further explore, without digging further into the other 3 models. 